# Mock Programming Exam
© Kajetan Knopp 2022

Remember to include typehints and docstrings whenever needed.

Run this cell before starting:

In [1]:
import sys
import csv
import re
import string
import random
from typing import Any, Dict, Generator, Iterable, List, Set, Tuple
sys.setrecursionlimit(30000)

### 1. List comprehensions

Write a function that takes in a string and returns that string with each letter (a-z) changed to the next one in the alphabet (so a->b, f->g, etc.). Write at least one test for this function.

##### Assumptions:
1. All letters are lowercase
2. 'z' should be changed to 'a'
3. The input string consists of all possible characters (except uppercase letters)

In [2]:
# Your code here:
def higherLetters(input_string : str) -> str:
    return "".join(['a' if letter=='z' else chr(ord(letter)+1) for letter in input_string if letter.isalpha()])

higherLetters("abcdz12454berc")

'bcdeacfsd'

### 2. OOP (Object-Oriented Programming)

You are asked to help in in an online store. It sells several different items, however, the owners want you to take a look at the books the store offers. Books are becoming less popular and take up too much space so they want to get rid of some of them! Here's what you need to do:

a) Create a class and call it "Item". It will describe the items being sold. Create a class attribute called "Warehouse", which can be set to "Eindhoven" or "Tilburg". Initialize this object with "price" equal to 10.

In [3]:
class Item:
    Warehouse = ["Eindhoven", "Tilburg"]
    
    def __init__(self, price=10):
        self.price = price

b) Now create a class "Book", which inherits from "Item". Create these following fields in the constructor:
<ul>
    <li>title (str)</li>
    <li>author (str)</li>
    <li>ISBN (int)</li>
    <li>genre (str)</li>
    <li>price (int)</li>
    <li>warehouse (int) : 0 - Eindhoven, 1 - Tilburg</li>
    <li>reviews (list[int])</li>
</ul>
Remember to create a readable representation of your class!

In [4]:
class Book(Item):
    def __init__(self, title, author, ISBN, genre, price, warehouse):
        super().__init__(int(price))
        self.title = title
        self.author = author
        self.ISBN = int(ISBN)
        self.genre = genre
        self.warehouse = warehouse
        self.reviews = []
        
    def __str__(self):
        return f"{self.ISBN}, {self.title}"

c) Read in values from "books.csv" to class Book. Create a collection of these Book objects. The books of "Author 1" - "Author 25" are shipped from Eindhoven, while the rest is shipped from Tilburg.

In [5]:
book_dict = {}

def read_book_csv(file):
    with open(file) as books:
        reader = csv.DictReader(books)
        for line in reader:
            if(int(re.findall(r'[0-9]+', line["Author"])[0]) <= 25):
                warehouse = 0
            else:
                warehouse = 1
            book_dict[int(line["ISBN"])] = Book(line["Title"], line["Author"], line["ISBN"], line["Genre"], line["Price"], warehouse)
            
read_book_csv("books.csv")

d) Now create a class called "Review", where user reviews will be held. Populate the book list with corresponding Review classes. You will find the info regarding reviews in "reviews.csv". Notice that some reviews are coded wrong - clean them up using regex.

In [6]:
def readReviews(file):
    review_list = []
    with open(file) as reviews:
        reader = csv.DictReader(reviews)
        for line in reader:
            review_list.append([line["ISBN"], line["Grade"]])
    return review_list

def populateBooks(review_list):
    for review in review_list:
        book_dict[int(review[0])].reviews.append(int(review[1]))
            
populateBooks(readReviews("reviews.csv"))

e) Answer the following questions (create at least one function that returns the result for each of them). Try to use at least one self-defined class method.
<ol>
    <li>Which book(s) has the highest average grade?</li>
    <li>Which author(s) has the highest average grade?</li>
    <li>Which category(-ies) is the most expensive?</li>
    <li>Which warehouse has the better cost to grade ratio?</li>
</ol>

In [7]:
def getBestAverage():
    best_avg = 0
    for key, value in book_dict.items():
        avg = sum(value.reviews)/max(1, len(value.reviews))
        if(avg > best_avg):
            best_avg = avg
    
    return (best_avg, [value.title for _, value in book_dict.items() if sum(value.reviews)/
                       max(1, len(value.reviews)) == best_avg])

def getBestAuthor():
    authors = {}
    for key, value in book_dict.items():
        avg = sum(value.reviews)/max(1, len(value.reviews))
        if(value.author in authors):
            authors[value.author].append(avg)
        else:
            authors[value.author] = [avg]
    
    best_avg = 0
    for author, value in authors.items():
        avg = sum(value)/max(1, len(value))
        if(avg > best_avg):
            best_avg = avg
    
    return (best_avg, [key for key, value in authors.items() if sum(value)/
                       max(1, len(value)) == best_avg])

def mostExpensiveCategory():
    categories = {}
    for key, value in book_dict.items():
        if(value.genre in categories):
            categories[value.genre].append(value.price)
        else:
            categories[value.genre] = [value.price]
    
    best_avg = 0
    for category, value in categories.items():
        avg = sum(value)/max(1, len(value))
        if(avg > best_avg):
            best_avg = avg
    
    return (best_avg, [key for key, value in categories.items() if sum(value)/
                       max(1, len(value)) == best_avg])

def betterRatio():
    warehouses = {}
    for key, value in book_dict.items():
        if(value.warehouse in warehouses):
            warehouses[value.warehouse].append(sum(value.reviews)/max(1, len(value.reviews))/value.price)
        else:
            warehouses[value.warehouse] = [sum(value.reviews)/max(1, len(value.reviews))/value.price]
    
    for warehouse, value in warehouses.items():
        avg = sum(value)/max(1, len(value))
        print(warehouse, avg)

print(getBestAverage())
print(getBestAuthor())
print(mostExpensiveCategory())
print(betterRatio())

(8.333333333333334, ['Book 573'])
(5.815554782519068, ['Author 39'])
(56.833333333333336, ['Science fiction'])
0 0.21930988177994337
1 0.2869497986873434
None


f) Now, using the data from (e) create a descriptive analysis of the results. Which warehouse should the owners close?

#### Your analysis:
The best score of a book is 8.3 for Book 573. The best author is Author 39 with 5.8. The most expensive genre is science fiction with 56.8 euros. The better warehouse is Tilburg.

### 3. Algorithm analysis

Take a look at algorithms (a) and (b). Answer the questions below their descriptions.

#### Algorithm (a):

In [8]:
def f(number, guess=1.0):
    if(abs(guess*guess-number)<0.05):
        return guess
    return (number, guess+(number-guess*guess)/2)

- What does this algorithm do? What is it's result?
##### Answer: 

- Why does it stop working for certain values?
##### Answer:

- Which programming concept(s) is used in this algorithm?
##### Answer:

- Write <b>typehints</b> and a <b>docstring</b> for this function!

#### Algorithm (b)

In [9]:
def mystery(A):
    for i in range(3):
        k = i
        for j in range(i+1, len(A)):
            if A[k] < A[j]:
                k = j         
        A[i], A[k] = A[k], A[i]
    return A

- What does this algorithm do? What is the result?
##### Answer:

- How could it be improved? What can be done to make it more readable?
###### Answer:

- Design a set of doctests and add comments to the code.

### 4. General Programming

a) Write a function that sanitizes a given string and seperates it into a list of words (delimited by spaces). Make each element of the list start with a capital letter. Write 3 tests for this function.

In [10]:
def separate(input_string):
    input_string = input_string.strip()
    words = re.findall(r'[A-Za-z]+', input_string)
    wordlist = []
    for word in words:
        wordlist.append(word.capitalize())
    return wordlist

separate("  !@#$122415siema eniu!!313 co tam slucahac  ")

['Siema', 'Eniu', 'Co', 'Tam', 'Slucahac']

b) Implement a stack. A stack is a data structure where the last inputted element is the first element to be read and removed.
Example of operations on a stack:
- empty list -> []
- add 2 -> [2]
- add 5 -> [2,5]
- read -> [2]
- add 6 -> [2,6]
- add 10 -> [2,6,10]
- read -> [2,6]

Given a set of operations (inputted as a string, denoted by 'a' - add element and 'r' - read/remove element) determine the final state of the stack.

Example of input: "a3ra4a7a2rr" (add 3, read, add 4, add 7, add 2, read, read) -> [4]

In [11]:
def stack(operations, state = []):
    i = 0
    while i < len(operations):
        if(operations[i] == 'a'):
            state.append(int(operations[i+1]))
            i+=2
        if(operations[i] == 'r'):
            print(state[-1])
            del state[-1]
            i+=1
    return state
stack("a3ra4a7a2rr")

3
2
7


[4]

c) Generate 15 tuples where each tuple consists of a random number from the range [0, 100] and a second number, which is the index (so t indicates how many tuples appeared before). Use a generator.

For size 2 the result would be: ((55, 0), (12, 1)).

In [16]:
result = ((random.randrange(0,101), i) for i in range(0,15))

for i in result:
    print(i)

(57, 0)
(27, 1)
(85, 2)
(6, 3)
(40, 4)
(79, 5)
(97, 6)
(65, 7)
(31, 8)
(68, 9)
(0, 10)
(87, 11)
(77, 12)
(79, 13)
(5, 14)


Now print each element twice. Why doesn't that work? What's wrong?
##### Answer:

### 5. Theory

a) What are the main differences between sets, lists, dictionaries and tuples? Compare these DS and give at least one unique feature of each.
##### Answer:

b) Describe what the methods zip and enumerate do.
##### Answer:

c) Describe how a class is structured - what 2 main methods are often used, which values differ between instances and which ones remain the same?
##### Answer:

d) What happens when one class inherits from another and both have a function with the same name? What about the other functions?
##### Answer: